In [1]:
### Import Packages
import pandas as pd

#### Questions and ground_truths


In [2]:
#df_benachmark = pd.read_json('../../DATA/RAG-evaluation/ARAGOG-main/eval_questions/benchmark.json')

df_benachmark = pd.read_csv("./benchmark-security2.csv", index_col=False)
df_benachmark.head()

,question,contexts,ground_truth,evolution_type,metadata,episode_done
0,How do LLMs' attention limits affect their rol...,[' al. [2024]\nCreativity is a critical asset ...,"LLMs typically lack situational awareness, whi...",multi_context,"[{'Published': '2024-05-09', 'Title': 'Artific...",True
1,What is the performance of PENTESTGPT in the p...,"['\ndesign, which retains the full testing con...",PENTESTGPT's performance in the picoMini CTF c...,simple,"[{'Published': '2024-06-02', 'Title': 'Pentest...",True
2,How do auto-grade MCUs and remote access speed...,['aims to discover as much information about t...,Auto-grade MCUs and remote access speed up ECU...,reasoning,"[{'Published': '2024-04-02', 'Title': 'Towards...",True
3,How does the critical load shedding threshold ...,['Critically \nStable\nAfter \nfailed \nUVLS\n...,The critical load shedding threshold is the mi...,multi_context,"[{'Published': '2024-05-26', 'Title': 'Make Sa...",True
4,How do chosen LLMs fare on pen testing benchma...,[' to assess the performances of various LLMs ...,The chosen LLMs demonstrate proficiency in man...,multi_context,"[{'Published': '2024-06-02', 'Title': 'Pentest...",True


In [3]:
len(df_benachmark)

50

#### RAG Documents


In [5]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

In [6]:
from  langchain.schema import Document
import json
from typing import Iterable
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.utils import filter_complex_metadata

def clean_metadata(metadata):
    if isinstance(metadata, str):
        # If metadata is a string, return it as is
        return metadata
    elif isinstance(metadata, dict):
        # If it's a dictionary, filter out None values and non-simple types
        return {k: v for k, v in metadata.items() if v is not None and isinstance(v, (str, int, float, bool))}
    else:
        # If it's neither a string nor a dictionary, return an empty dict
        return {}

def load_docs_from_jsonl(file_path) -> Iterable[Document]:
    array = []
    with open(file_path, 'r') as jsonl_file:
        for line in jsonl_file:
            data = json.loads(line)
            
            # Clean the metadata if it exists
            if 'metadata' in data:
                data['metadata'] = clean_metadata(data['metadata'])
            
            # Create Document object
            obj = Document(
                page_content=data.get('page_content', ''),
                metadata=data.get('metadata', {})
            )
            array.append(obj)
    return array
    
docs_benchmark = load_docs_from_jsonl('./security_dataset2.jsonl')

# Create a text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]
)

# Split the documents
split_docs = text_splitter.split_documents(docs_benchmark)

print(f"Number of original documents: {len(docs_benchmark)}")
print(f"Number of split documents: {len(split_docs)}")

Number of original documents: 11
Number of split documents: 483


In [7]:
#len(docs_benchmark)
#split_docs
docs_benchmark = split_docs

In [8]:
#docs_benchmark

In [9]:
#from langchain_community.vectorstores import FAISS
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
### embedding model
model = "sentence-transformers/multi-qa-MiniLM-L6-cos-v1"
### embedding object
embeddings = HuggingFaceEmbeddings(model_name = model)
### document embeddings
db_benchmark = Chroma.from_documents(docs_benchmark, embeddings)

/home/hamzicd/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/hamzicd/.local/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/hamzicd/.local/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


#### Select LLM Models&Versions on Replicate & OpenAI & Anthrope

In [10]:
import replicate
from dotenv import load_dotenv
import os

load_dotenv()

### Replicate
os.environ["REPLICATE_API_TOKEN"] = os.getenv("REPLICATE_API_TOKEN")
### OpenAI
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
### Anthropic
os.environ["ANTHROPIC_API_KEY"] = os.getenv("ANTHROPIC_API_KEY")
### Gemini pro (Google)
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")


In [32]:

models = [
    ### OpenAI
    #"gpt-3.5-turbo",
    #"gpt-4",
    #"gpt-4-turbo",
    #"gpt-4o",
    
    ### Anthropic
    #"claude-3-opus-20240229",
    #"claude-3-5-sonnet-20240620",

    ### Google
    #"gemini-1.0-pro",
    #"gemini-1.5-pro",
    #"gemini-1.5-flash",
    
    ### replicate models
    #"meta/meta-llama-3-70b-instruct",
    #"meta/meta-llama-3-8b",
    
    #"meta/meta-llama-3-8b-instruct", # ERROR : No Version matches the given query.
    #"mistralai/mixtral-8x7b-instruct-v0.1",
    #"meta/llama-2-7b-chat",
    #"meta/llama-2-70b-chat",
    #"meta/llama-2-13b-chat",
    #"mistralai/mistral-7b-instruct-v0.2",
    #"mistralai/mistral-7b-v0.1",
    #"mistralai/mistral-7b-instruct-v0.1",
    #"replicate/dolly-v2-12b", # ERROR : This version has been disabled because it consistently fails to complete setup.
    #"meta/meta-llama-3-70b",
    #"01-ai/yi-34b-chat",
    #"replicate/vicuna-13b", # ERROR: This version has been disabled because it consistently fails to complete setup.
    #"01-ai/yi-6b",
    # "replicate/flan-t5-xl", # NO BENCHMARKS
    #"stability-ai/stablelm-tuned-alpha-7b", # ERROR: This version has been disabled because it consistently fails to complete setup.
    #"replicate/llama-7b", # ERROR: This version has been disabled because it consistently fails to complete setup.
    #"google-deepmind/gemma-2b-it",
    #"google-deepmind/gemma-7b-it",
    #"nateraw/nous-hermes-2-solar-10.7b",
    #"replicate/oasst-sft-1-pythia-12b", # ERROR: This version has been disabled because it consistently fails to complete setup.
    #"replicate/gpt-j-6b",
    #"google-deepmind/gemma-7b",
    #"01-ai/yi-6b-chat",
    #"lucataco/phi-2", # ERROR: You must agree to use this model for research-only, you cannot use this model comercially.
    #"google-deepmind/gemma-2b",
    "lucataco/qwen1.5-14b", # ERROR: Temperature Must be greater than or equal to 0.1
    "lucataco/qwen1.5-7b",
    
    #"lucataco/olmo-7b", # NO BENCHMARKS # ERROR: This version has been disabled because it consistently fails to complete setup.
    #"adirik/mamba-1.4b", # NO BENCHMARKS
    #"adirik/mamba-2.8b-slimpj", # NO BENCHMARKS
    #"adirik/mamba-370m", # NO BENCHMARKS   
    #"adirik/mamba-790m", # NO BENCHMARKS
]

### get versions of models
models_versions = []
for m in models:
    # check if replicate model
    if "/" in m:
        print(m)
        model = replicate.models.get(m)
        models_versions.append(m+":"+model.default_example.version)
    # else no version
    else:
        models_versions.append(m+":"+"0")

lucataco/qwen1.5-14b
lucataco/qwen1.5-7b


In [33]:
### model:version
models_versions

['lucataco/qwen1.5-14b:28c4bbc17ee1575bd2efe2d805a6c3da9f555bf6298d447d9d8d8ebfb891c4a1',
 'lucataco/qwen1.5-7b:f85bec5b21ba0860e0f200be6ef5af9d5a65b974b9f99e36eb036d21eab884de']

In [19]:
#replicate.models.get("meta/meta-llama-3-8b-instruct")

### RAG Setup

In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

template = """Answer the question based only on the following context:

{context}

Question: {question}
"""
#retriever = db_benchmark.as_retriever(search_type="mmr", search_kwargs={'k': 2, 'fetch_k': 10})
retriever = db_benchmark.as_retriever(search_type="similarity", search_kwargs={'k': 2})
prompt = ChatPromptTemplate.from_template(template)

### create context
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

In [14]:
### use for testing purposes
#df_benachmark = df_benachmark.head(3)

### LLM Judge

In [15]:
from langchain_community.llms import Replicate
from langchain_openai import ChatOpenAI

#llm_evaluator = Replicate(
#    model="meta/meta-llama-3-70b-instruct:fbfb20b472b2f3bdd101412a9f70a0ed4fc0ced78a77ff00970ee7a2383c575d",
#    model_kwargs={"temperature": 0.01, "max_length": 500, "top_p": 1},
#)

llm_evaluator = ChatOpenAI(temperature=0, model_name="gpt-4o", top_p=1)

/home/hamzicd/.local/lib/python3.10/site-packages/langchain_core/utils/utils.py:161: UserWarning: WARNING! top_p is not default parameter.
                top_p was transferred to model_kwargs.
                Please confirm that top_p is what you intended.
  warnings.warn(


### Run Questions on LLMs

In [34]:
from datasets import Dataset 
from langchain_anthropic import ChatAnthropic
from collections import defaultdict
from langchain_core.runnables import RunnableParallel
from langchain_google_vertexai import ChatVertexAI

from ragas import evaluate

from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    answer_correctness,
    answer_similarity,
)

### for every model & version
for model_version in models_versions:
    print(model_version)

    ### Google Gemeni
    if model_version.startswith("gemini"):
        #llm = ChatVertexAI(model_name=model_version.split(":")[0], temperature=0, project="gen-lang-client-0636245129")
        llm = ChatVertexAI(model_name=model_version.split(":")[0], temperature=0, project="227230228998")
    ### OpenAI
    elif model_version.startswith("gpt"):
        llm = ChatOpenAI(temperature=0, model_name=model_version.split(":")[0], top_p=1)
    ### Anthropic Claude
    elif model_version.startswith("claude"):
        llm = ChatAnthropic(temperature=0, model=model_version.split(":")[0], top_p=1)
    ### Open Source    
    else:
        llm = Replicate(
            model=model_version,
            model_kwargs={"temperature": 0.1, "max_length": 500, "top_p": 1},
            # phi 3 -> temperature min. 0.1
            # qwen -> temperature min. 0.1
            # all others -> 0.01
        )
    
    
    print("llm initialized...")
    rag_chain_from_docs = (
        RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
        | prompt
        | llm
        | StrOutputParser()
    )
    print("chain initialized...")
    rag_chain_with_source = RunnableParallel(
        {"context": retriever, "question": RunnablePassthrough()}
    ).assign(answer=rag_chain_from_docs)
    
    print("Q&A evaluation initialized...")

    model_name = model_version.split(":")[0]
    print(model_name)

    ansObj = None
    data_samples = defaultdict(list)
    print(f"Using LLM: {type(llm).__name__}")

    ######################################
    ### for every Question in Benchmark
    ######################################
    for index, row in df_benachmark.iterrows():
        try:
            #print(row['questions'], row['ground_truths'])
            ### question
            question = row['question']
            ### ground truths
            ground_truth = row['ground_truth']
        
            ### call to LLM
            print("calling LLM, question: ", index)
            ansObj = rag_chain_with_source.invoke(question)
            ### answer
            answer = ansObj["answer"]
            ### context list
            contextList = []
            for doc in ansObj["context"]:
                #print(doc.page_content)
                contextList.append(doc.page_content)
        
            
            data_samples["question"].append(question)
            data_samples["answer"].append(answer)
            data_samples["contexts"].append(contextList)
            data_samples["ground_truth"].append(ground_truth)
            print("Q:", question)
            print("A:", answer)
            print("GT:", ground_truth)#
            print("---------------")
        except Exception as e:
            print("The error is: ",e)
            
        #if index > 3:
        #    break
    
    dataset = Dataset.from_dict(data_samples)
    new_column = [model_name] * len(dataset)
    dataset = dataset.add_column("model", new_column)
    dataset.to_csv("./DATA/output/dataset2/"+ model_name+"-EVALUATION-DF.csv", encoding='utf-8', index=False)

    ### evaluate
    result = evaluate(
        dataset,#amnesty_qa["eval"],
        metrics=[
            context_precision,
            faithfulness,
            answer_relevancy,
            context_recall,
            answer_correctness,
            answer_similarity,
        ],
        llm=llm_evaluator,
        embeddings=embeddings,
    )

    ### print result
    print(model_name + ", result: ", result)
    eval_result_df = result.to_pandas()
    eval_result_df["model"] = model_name
    eval_result_df.to_csv("./DATA/output/dataset2/"+model_name+"-RESULTS-DF.csv", encoding='utf-8', index=False)

    print("######################################")
    print("######################################")
    print("######################################")

lucataco/qwen1.5-14b:28c4bbc17ee1575bd2efe2d805a6c3da9f555bf6298d447d9d8d8ebfb891c4a1
llm initialized...
chain initialized...
Q&A evaluation initialized...
lucataco/qwen1.5-14b
Using LLM: Replicate
calling LLM, question:  0
Q: How do LLMs' attention limits affect their role in dynamic security tasks, despite tools like PentestGPT?
A: LLMs' attention and situational awareness limitations significantly hinder their effectiveness in dynamic security tasks, such as penetration testing. They struggle to maintain a coherent understanding of the broader testing scenario, which leads to losing sight of earlier findings and applying reasoning consistently. LLMs tend to overemphasize recent tasks and neglect other potential attack surfaces, especially if they are not directly related to the current conversation. This real-time decision-making and adaptability deficiency is crucial in offensive security, where context and rapid analysis are vital. Tools like PentestGPT can assist in specific sub-

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Evaluating:   0%|          | 0/300 [00:00<?, ?it/s]

lucataco/qwen1.5-14b, result:  {'context_precision': 0.5500, 'faithfulness': 0.8090, 'answer_relevancy': 0.6915, 'context_recall': 0.5110, 'answer_correctness': 0.4396, 'answer_similarity': 0.8309}
######################################
######################################
######################################
lucataco/qwen1.5-7b:f85bec5b21ba0860e0f200be6ef5af9d5a65b974b9f99e36eb036d21eab884de
llm initialized...
chain initialized...
Q&A evaluation initialized...
lucataco/qwen1.5-7b
Using LLM: Replicate
calling LLM, question:  0
Q: How do LLMs' attention limits affect their role in dynamic security tasks, despite tools like PentestGPT?
A: LLMs, particularly in the context of dynamic security tasks like penetration testing, are hindered by their inability to maintain situational awareness. They lack the understanding and adaptability that humans possess, which is crucial in real-time decision-making and adapting to unforeseen circumstances. This limitation affects their effectiveness,

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Evaluating:   0%|          | 0/300 [00:00<?, ?it/s]

lucataco/qwen1.5-7b, result:  {'context_precision': 0.5500, 'faithfulness': 0.6727, 'answer_relevancy': 0.7602, 'context_recall': 0.5221, 'answer_correctness': 0.4279, 'answer_similarity': 0.8278}
######################################
######################################
######################################
